In [29]:
%load_ext lab_black
import pandas as pd
import numpy as np
import mplfinance as fplt
import talib
import datetime
from dateutil.relativedelta import relativedelta, TH
import warnings
from tqdm import tqdm

tqdm.pandas()

warnings.simplefilter(action="ignore", category=FutureWarning)

%matplotlib qt
data = pd.read_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"
)

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [30]:
data["Date"] = data["Date"].progress_apply(
    lambda x: datetime.datetime.strptime(str(x), "%Y%m%d")
)
data = data.set_index("Date")

100%|██████████| 1903131/1903131 [00:18<00:00, 102924.13it/s]


In [31]:
data.columns

Index(['Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [32]:
nse500 = pd.read_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\WatchList\Nifty500.tls",
    header=None,
)
nse500.columns = ["symbol"]

In [38]:
start_date = datetime.date.today() - relativedelta(months=3)
end_date = datetime.date.today()
dfDf = pd.DataFrame()


def findDivergence(symbol):
    global data, start_date, end_date, dfDf
    stkData = data.loc[
        (
            (data.Ticker == symbol)
            & (data.index.date >= start_date)
            & (data.index.date <= end_date)
        ),
        ["Close", "High", "Low", "Open", "Volume", "Ticker"],
    ].dropna()  # & (data.index.Date<=datetime.date(2022,3,9))
    if stkData.shape[0] > 30:
        stkData["rsi"] = talib.RSI(stkData["Close"], timeperiod=3)
        indx = 0  # previous index
        stkData["lower"] = 0
        stkData["higher"] = 0
        stkData["lowerRsi"] = 0
        stkData["higherRsi"] = 0

        lowIndx = list(stkData.columns).index("lower")
        highIndx = list(stkData.columns).index("higher")
        lowRsiIndx = list(stkData.columns).index("lowerRsi")
        highRsiIndx = list(stkData.columns).index("higherRsi")
        rsiIndx = list(stkData.columns).index("rsi")

        for _, row in stkData.iterrows():
            if indx == 0:
                stkData.iloc[indx, lowIndx] = np.nan
                stkData.iloc[indx, highIndx] = np.nan
                stkData.iloc[indx, lowRsiIndx] = np.nan
                stkData.iloc[indx, highRsiIndx] = np.nan
                indx += 1
                # print("c")
                continue
            elif indx == (stkData.shape[0] - 1):
                stkData.iloc[indx, lowIndx] = row["Close"]
                stkData.iloc[indx, highIndx] = row["Close"]
                stkData.iloc[indx, lowRsiIndx] = row["rsi"]
                stkData.iloc[indx, highRsiIndx] = row["rsi"]
                indx += 1
                # print("c")
                continue
            else:
                if (
                    row["Close"] < stkData.iloc[indx - 1, 0]
                    and row["Close"] < stkData.iloc[indx + 1, 0]
                ):
                    stkData.iloc[indx, lowIndx] = row["Close"]
                    stkData.iloc[indx, highIndx] = np.nan
                elif (
                    row["Close"] > stkData.iloc[indx - 1, 0]
                    and row["Close"] > stkData.iloc[indx + 1, 0]
                ):
                    stkData.iloc[indx, highIndx] = row["Close"]
                    stkData.iloc[indx, lowIndx] = np.nan
                else:
                    stkData.iloc[indx, highIndx] = np.nan
                    stkData.iloc[indx, lowIndx] = np.nan

                if (
                    row["rsi"] < stkData.iloc[indx - 1, rsiIndx]
                    and row["rsi"] < stkData.iloc[indx + 1, rsiIndx]
                ):
                    stkData.iloc[indx, lowRsiIndx] = row["rsi"]
                    stkData.iloc[indx, highRsiIndx] = np.nan
                elif (
                    row["rsi"] > stkData.iloc[indx - 1, rsiIndx]
                    and row["rsi"] > stkData.iloc[indx + 1, rsiIndx]
                ):
                    stkData.iloc[indx, highRsiIndx] = row["rsi"]
                    stkData.iloc[indx, lowRsiIndx] = np.nan
                else:
                    stkData.iloc[indx, highRsiIndx] = np.nan
                    stkData.iloc[indx, lowRsiIndx] = np.nan

                indx += 1
        temp1 = stkData.dropna(subset=["lower", "lowerRsi"], how="any")
        temp1.loc[:, "chang_lower"] = temp1["lower"].pct_change(1)
        temp1.loc[:, "chang_Rsi_lower"] = temp1["lowerRsi"].pct_change(1)
        temp1.loc[:, "cls_bull_div"] = temp1.apply(
            lambda x: x["Close"]
            if x["chang_lower"] < 0 and x["chang_Rsi_lower"] > 0
            else np.nan,
            axis=1,
        )
        stkData = pd.merge(
            left=stkData.reset_index(),
            right=temp1[["cls_bull_div"]].reset_index(),
            left_on="Date",
            right_on="Date",
            how="left",
        ).set_index("Date")
        temp1 = stkData.dropna(subset=["higher", "higherRsi"], how="any")
        temp1.loc[:, "chang_upper"] = temp1["higher"].pct_change(1)
        temp1.loc[:, "chang_Rsi_upper"] = temp1["higherRsi"].pct_change(1)
        temp1.loc[:, "cls_bear_div"] = temp1.apply(
            lambda x: x["Close"]
            if x["chang_upper"] > 0 and x["chang_Rsi_upper"] < 0
            else np.nan,
            axis=1,
        )
        stkData = pd.merge(
            left=stkData.reset_index(),
            right=temp1[["cls_bear_div"]].reset_index(),
            left_on="Date",
            right_on="Date",
            how="left",
        ).set_index("Date")
        dfDf = dfDf.append(stkData)


nse500.symbol.progress_apply(lambda symbol: findDivergence(symbol))

  0%|          | 0/501 [00:00<?, ?it/s]C:\Users\ksdee\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\ksdee\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ksdee\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

0      None
1      None
2      None
3      None
4      None
       ... 
496    None
497    None
498    None
499    None
500    None
Name: symbol, Length: 501, dtype: object

c
c


/Users/shubhangi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shubhangi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [68]:
end_date = datetime.date(2022, 3, 25)
start_date = end_date - relativedelta(weeks=1)

dfDf.loc[
    ((dfDf.index.date >= start_date) & (dfDf.index.date < end_date)),
    ["Ticker", "cls_bull_div"],
].dropna().sort_index()

,Ticker,cls_bull_div
Date,,
2022-03-21,BLUESTARCO,978.750000
2022-03-21,HGS,1092.250000
2022-03-21,LTI,5992.399902
2022-03-22,IDFCFIRSTB,40.599998
2022-03-22,PETRONET,194.699997
2022-03-23,JSWENERGY,298.149994
2022-03-23,MAHINDCIE,166.949997
2022-03-23,PGHL,4260.600098
2022-03-23,SHILPAMED,401.750000


In [78]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
temp = dfDf.loc[dfDf.Ticker == "PETRONET"]
style = fplt.make_mpf_style(
    marketcolors=fplt.make_marketcolors(up="green", down="red", inherit=True),
    gridcolor="gray",
    gridstyle="--",
    gridaxis="both",
)

sub_plots = [fplt.make_addplot(temp["lower"], panel=0, type="scatter", color="black")]
if temp["cls_bull_div"].dropna().shape[0] > 0:
    sub_plots.append(
        fplt.make_addplot(
            temp["cls_bull_div"],
            panel=0,
            type="scatter",
            marker="o",
            markersize=100,
            color="yellow",
        )
    )
if temp["cls_bear_div"].dropna().shape[0] > 0:
    sub_plots.append(
        fplt.make_addplot(
            temp["cls_bear_div"],
            panel=0,
            type="scatter",
            marker="o",
            markersize=100,
            color="pink",
        )
    )
sub_plots.append(
    fplt.make_addplot(temp["higher"], panel=0, type="scatter", color="blue")
)
sub_plots.append(
    fplt.make_addplot(
        temp["rsi"],
        type="line",
        linestyle="dashed",
        panel=2,
        color="black",
        ylabel="rsi",
    )
)
sub_plots.append(
    fplt.make_addplot(
        temp["lowerRsi"],
        type="scatter",
        marker="o",
        panel=2,
        color="red",
        ylabel="rsi",
        markersize=20,
    )
)
sub_plots.append(
    fplt.make_addplot(
        temp["higherRsi"],
        type="scatter",
        marker="o",
        panel=2,
        color="green",
        ylabel="rsi",
        markersize=20,
    )
)


fig, axes = fplt.plot(
    temp, type="candle", style=style, addplot=sub_plots, volume=True, returnfig=True,
)

In [53]:
temp

,Close,High,Low,Open,Volume,Ticker,rsi,lower,higher,lowerRsi,higherRsi,cls_bull_div,cls_bear_div
Date,,,,,,,,,,,,,
2021-12-27,1013.700012,1024.949951,1001.150024,1009.950012,78877.0,THYROCARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-28,1014.799988,1023.950012,1005.750000,1018.799988,40682.0,THYROCARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-29,1072.900024,1080.750000,1017.000000,1020.799988,309729.0,THYROCARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-30,1109.750000,1115.000000,1069.599976,1072.000000,246043.0,THYROCARE,100.000000,NaN,1109.750000,NaN,NaN,NaN,NaN
2021-12-31,1107.550049,1121.000000,1095.199951,1112.000000,101983.0,THYROCARE,96.678481,1107.550049,NaN,96.678481,NaN,NaN,NaN
2022-01-03,1111.900024,1132.000000,1102.099976,1109.800049,97772.0,THYROCARE,96.976355,NaN,1111.900024,NaN,96.976355,NaN,NaN
2022-01-04,1093.599976,1119.000000,1089.099976,1119.000000,97675.0,THYROCARE,61.929443,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,1084.599976,1108.000000,1070.000000,1102.000000,125775.0,THYROCARE,48.894114,1084.599976,NaN,48.894114,NaN,NaN,NaN
2022-01-06,1086.550049,1108.000000,1068.800049,1080.000000,98107.0,THYROCARE,52.166444,NaN,1086.550049,NaN,52.166444,NaN,NaN
